# The 10-Minute Modeling Tool

Let's build a modeling tool in 10 minutes!

We'll use [Neo4j](http://neo4j.com/) for data storage, [Python](https://www.python.org/) for data processing, [vis.js](http://visjs.org/) for data visualization.

## Neo4j

From [Wikipedia's Neo4j article](https://en.wikipedia.org/wiki/Neo4j) (with ephasis by me):

> Neo4j is an **open-source graph database** implemented in Java and **accessible from software written in other languages** using the **Cypher query language** through a transactional HTTP endpoint. The developers describe Neo4j as an ACID-compliant transactional database with native graph storage and processing. **Neo4j is the most popular graph database.**

A graph database is simply a set of **nodes** (things) and **relationships** (pairs of things). Both nodes and relationships can have labels or properties associated with them. Graph-traversal algorithms (both built-in and custom-built) read the properties of nodes and traverse the relationships between them to return nodes and relationships of interest.

## 10-second overview

In this session, we'll connect to the blank Neo4j database running on our local server.

Then, we'll populate it with nodes that will define:

* a "model element"
* a "model connection"
* our metamodel \*
* our project model
* our project data

and relationships that will define:

* how our metamodel \* conforms to the idea of "elements" and "connections"
* how our project model conforms to our metamodel \*
* how our project data conforms to our project model

Finally, we'll query our database and obtain useful views of our data, as well as suggestions for what additional data could be defined (what to ask our stakeholders).

\* We'll use a sloppy subset of UML in this session, but any metamodel is possible - SysML, UML, multi-layed ontologies, etc.

## Jack and Jill and Kidney Failure

Jack needs a kidney transplant or he will die. We will find him a donor with modeling!

## Connect to the Neo4j Database

First we'll connect to the database and delete any existing nodes and relationships.

We'll communicate with Neo4j through the `py2neo` Python library.

In [1]:
from py2neo import authenticate, Graph
from py2neo.error import Unauthorized

graph = Graph()

try:
    graph.delete_all()
except Unauthorized:
    # If you have a username/password, replace neo4j/password with your username/password
    authenticate('localhost:7474', 'neo4j', 'password')
    graph.delete_all()

## Element and Connection

Modeling is all about elements and connections. Similarly, graphs are all about nodes and relationships.

We'll draw a clear distinction between element/connection and node/relationship right now - **both elements and connections are nodes.**

##### Connections are Nodes?

Connections have a lot of information in them - a connection's source and target are specific elements, and its generalization and specializations are other specific connections.

Because a connection has relationships to those elements and connections, it cannot be a relationship itself. **Relationships do not have relationships.**

#### Let's define our first nodes!

In [2]:
from py2neo import Node
from scripts.vis import draw

# Neo4j nodes are defined by labels and properties
element = Node(
    'Element',      # label
    name='element'  # property
)

# This saves the node to the graph database
graph.create(element)

connection = Node('Connection', name='connection')
graph.create(connection)

# This function draws all of the graph's nodes and relationships in a bouncy vis.js visualization - drag the nodes around!
draw(graph)

Elements connect to other elements through connections, so elements are the sources and targets of connections.

We'll define relationships with a subject, verb, and object.

In the below cell, the relationships are **an element is the source of a connection** and **an element is the target of a connection**.

In [3]:
from py2neo import Relationship

# Neo4j relationships are mostly defined by a string label, as relationships do not inherit or relate to anything else.
IS_THE_SOURCE_OF = 'IS_THE_SOURCE_OF'
IS_THE_TARGET_OF = 'IS_THE_TARGET_OF'

graph.create(Relationship(element, IS_THE_SOURCE_OF, connection))
graph.create(Relationship(element, IS_THE_TARGET_OF, connection))

# Unfortunately, vis.js draws relationships on top of each other - "IS_THE_TARGET_OF" is underneath "IS_THE_SOURCE_OF".
draw(graph)

## Metamodel Elements

Now we need a metamodel - the logical framework that will eventually structure our model. For this project, we'll use a handful of elements and connections from UML, but the entirety of SysML can be defined in this same way.

We'll define Class, Abstract Class, and Interface

In [4]:
class_ = Node('Metamodel Element', name='class')
abstract_class = Node('Metamodel Element', name='abstract class')
interface = Node('Metamodel Element', name='interface')

graph.create(class_, abstract_class, interface)

draw(graph)

### Relating the metamodel

Class, Abstract Class, and Interface are all elements, so we relate the metamodel element nodes to the element node.

We'll use the relationship "X is a Y" to define a specific instance of a more abstract definition, much like "I am a Person".

*Note: If you see a distinction between inheritance, specialization, and instantiation, then A) bear with me and B) later you can define a sane metamodel.*

In [5]:
IS_A = 'IS_A'

graph.create(Relationship(class_,         IS_A, element))
graph.create(Relationship(abstract_class, IS_A, class_ ))
graph.create(Relationship(interface,      IS_A, element))

draw(graph)

The relationships "across layers" (between element/connection and the metamodel) are grayed-out.

## Metamodel Connections

The metamodel also contains connections between its elements, such as "a Class implements an Interface".

Let's create these connections and their relationships at the same time.

In [6]:
composes = Node('Metamodel Connection', name='composes')
graph.create(Relationship(composes, IS_A, connection))
graph.create(Relationship(class_, IS_THE_SOURCE_OF, composes))
graph.create(Relationship(class_, IS_THE_TARGET_OF, composes))

generalizes = Node('Metamodel Connection', name='generalizes')
graph.create(Relationship(generalizes, IS_A, connection))
graph.create(Relationship(class_, IS_THE_SOURCE_OF, generalizes))
graph.create(Relationship(class_, IS_THE_TARGET_OF, generalizes))

implements = Node('Metamodel Connection', name='implements')
graph.create(Relationship(implements, IS_A, connection))
graph.create(Relationship(class_,    IS_THE_SOURCE_OF, implements))
graph.create(Relationship(interface, IS_THE_TARGET_OF, implements))

draw(graph)

Drag nodes around to get a better view of the relationships.

## Model Elements

Let's define our project model - in order to find Jack a kidney donor, we'll need to model the following:

* humans
* kidneys
* blood types
* friendship
* dogs
* octopuses

Obviously we need to model dogs and octopuses in order to be thorough about this kidney search.

In [7]:
animal = Node('Model Element', name='animal')
graph.create(Relationship(animal, IS_A, abstract_class))

vertebrate = Node('Model Element', name='vertebrate')
graph.create(Relationship(vertebrate, IS_A, interface))

human = Node('Model Element', name='human')
graph.create(Relationship(human, IS_A, animal))

dog = Node('Model Element', name='dog')
graph.create(Relationship(dog, IS_A, animal))

octopus = Node('Model Element', name='octopus')
graph.create(Relationship(octopus, IS_A, animal))

organ = Node('Model Element', name='organ')
graph.create(Relationship(organ, IS_A, abstract_class))

kidney = Node('Model Element', name='kidney')
graph.create(Relationship(kidney, IS_A, organ))

blood_type = Node('Model Element', name='blood type')
graph.create(Relationship(blood_type, IS_A, class_))

draw(graph, labels=['Model Element'])

The above visualization is only a subset of the entire graph.

From now on, after adding to the graph, we'll view it in multiple stages, zooming out from the new nodes to see the entire graph.

Zoom in/out (by scrolling), pan (by dragging the white background), and drag nodes around to get a better view of all these graphs.

In [8]:
draw(graph)

## Model Connections

To define how these model elements connect, we not only **sub-type existing** connection elements, we also **create new** connection elements.

"A human is a type of animal" is a connection that we define for model completeness, but "a dog has a blood type" exists to enable specific dogs and specific blood types to connect when we input our dog and blood type data.

In [9]:
animal__generalizes__human = Node('Model Connection', name='animal generalizes human')
graph.create(Relationship(animal__generalizes__human, IS_A, generalizes))
graph.create(Relationship(animal, IS_THE_SOURCE_OF, animal__generalizes__human))
graph.create(Relationship(human,  IS_THE_TARGET_OF, animal__generalizes__human))

animal__generalizes__dog = Node('Model Connection', name='animal generalizes dog')
graph.create(Relationship(animal__generalizes__dog, IS_A, generalizes))
graph.create(Relationship(animal, IS_THE_SOURCE_OF, animal__generalizes__dog))
graph.create(Relationship(dog,    IS_THE_TARGET_OF, animal__generalizes__dog))

animal__generalizes__octopus = Node('Model Connection', name='animal generalizes octopus')
graph.create(Relationship(animal__generalizes__octopus, IS_A, generalizes))
graph.create(Relationship(animal,  IS_THE_SOURCE_OF, animal__generalizes__octopus))
graph.create(Relationship(octopus, IS_THE_TARGET_OF, animal__generalizes__octopus))

human__implements__vertebrate = Node('Model Connection', name='human implements vertebrate')
graph.create(Relationship(human__implements__vertebrate, IS_A, implements))
graph.create(Relationship(human,      IS_THE_SOURCE_OF, human__implements__vertebrate))
graph.create(Relationship(vertebrate, IS_THE_TARGET_OF, human__implements__vertebrate))

dog__implements__vertebrate = Node('Model Connection', name='dog implements vertebrate')
graph.create(Relationship(dog__implements__vertebrate, IS_A, implements))
graph.create(Relationship(dog,        IS_THE_SOURCE_OF, dog__implements__vertebrate))
graph.create(Relationship(vertebrate, IS_THE_TARGET_OF, dog__implements__vertebrate))

organ__composes__animal = Node('Model Connection', name='organ is inside')
graph.create(Relationship(organ__composes__animal, IS_A, composes))
graph.create(Relationship(animal, IS_THE_SOURCE_OF, organ__composes__animal))
graph.create(Relationship(organ,  IS_THE_TARGET_OF, organ__composes__animal))

has_blood_type = Node('Model Connection', name='has blood type')
graph.create(Relationship(has_blood_type, IS_A, composes))
graph.create(Relationship(animal,     IS_THE_SOURCE_OF, has_blood_type))
graph.create(Relationship(blood_type, IS_THE_TARGET_OF, has_blood_type))

kidney_is_inside = Node('Model Connection', name='kidney is inside')
graph.create(Relationship(kidney_is_inside, IS_A, composes))
graph.create(Relationship(kidney,     IS_THE_SOURCE_OF, kidney_is_inside))
graph.create(Relationship(vertebrate, IS_THE_TARGET_OF, kidney_is_inside))

is_friends_with = Node('Model Connection', name='is friends with')
graph.create(Relationship(is_friends_with, IS_A, connection))
graph.create(Relationship(animal, IS_THE_SOURCE_OF, is_friends_with))
graph.create(Relationship(animal, IS_THE_SOURCE_OF, is_friends_with))

draw(graph, labels=['Model Connection'])

In [10]:
draw(graph, relax_gray_relationships=True)

## Instance Elements

It's finally time for us to input data about Jack, his friends, and their kidneys!

In [11]:
jack = Node('Instance Element', name='jack')
graph.create(Relationship(jack, IS_A, human))

jill = Node('Instance Element', name='jill')
graph.create(Relationship(jill, IS_A, human))

john = Node('Instance Element', name='john')
graph.create(Relationship(john, IS_A, human))

jane = Node('Instance Element', name='jane')
graph.create(Relationship(jane, IS_A, human))

jasper = Node('Instance Element', name='jasper')
graph.create(Relationship(jasper, IS_A, dog))

jacoby = Node('Instance Element', name='jacoby')
graph.create(Relationship(jacoby, IS_A, octopus))

a_positive = Node('Instance Element', name='a+')
graph.create(Relationship(a_positive, IS_A, blood_type))

b_negative = Node('Instance Element', name='b-')
graph.create(Relationship(b_negative, IS_A, blood_type))

dea_one_point_one_negative = Node('Instance Element', name='dea 1.1-')
graph.create(Relationship(dea_one_point_one_negative, IS_A, blood_type))

invertebrate_blood = Node('Instance Element', name='invertebrate blood')
graph.create(Relationship(invertebrate_blood, IS_A, blood_type))

jack_left_kidney = Node('Instance Element', name="jack's left kidney")
graph.create(Relationship(jack_left_kidney, IS_A, kidney))

jill_right_kidney = Node('Instance Element', name="jill's right kidney")
graph.create(Relationship(jill_right_kidney, IS_A, kidney))

john_left_kidney = Node('Instance Element', name="john's left kidney")
graph.create(Relationship(john_left_kidney, IS_A, kidney))

john_right_kidney = Node('Instance Element', name="john's right kidney")
graph.create(Relationship(john_right_kidney, IS_A, kidney))

jane_left_kidney = Node('Instance Element', name="jane's left kidney")
graph.create(Relationship(jane_left_kidney, IS_A, kidney))

jane_right_kidney = Node('Instance Element', name="jane's right kidney")
graph.create(Relationship(jane_right_kidney, IS_A, kidney))

jasper_left_kidney = Node('Instance Element', name="jasper's left kidney")
graph.create(Relationship(jasper_left_kidney, IS_A, kidney))

jasper_right_kidney = Node('Instance Element', name="jasper's right kidney")
graph.create(Relationship(jasper_right_kidney, IS_A, kidney))

draw(graph, labels=['Instance Element'])

## Instance Connections

... and we connect the instance elements to complete the data...

In [12]:
jack__has_blood_type__a_positive = Node('Instance Connection', name='jack has blood type A+')
graph.create(Relationship(jack__has_blood_type__a_positive, IS_A, has_blood_type))
graph.create(Relationship(jack,       IS_THE_SOURCE_OF, jack__has_blood_type__a_positive))
graph.create(Relationship(a_positive, IS_THE_TARGET_OF, jack__has_blood_type__a_positive))

jill__has_blood_type__b_negative = Node('Instance Connection', name='jill has blood type B-')
graph.create(Relationship(jill__has_blood_type__b_negative, IS_A, has_blood_type))
graph.create(Relationship(jill,       IS_THE_SOURCE_OF, jill__has_blood_type__b_negative))
graph.create(Relationship(b_negative, IS_THE_TARGET_OF, jill__has_blood_type__b_negative))

jane__has_blood_type__a_positive = Node('Instance Connection', name='jane has blood type A+')
graph.create(Relationship(jane__has_blood_type__a_positive, IS_A, has_blood_type))
graph.create(Relationship(jane,       IS_THE_SOURCE_OF, jane__has_blood_type__a_positive))
graph.create(Relationship(a_positive, IS_THE_TARGET_OF, jane__has_blood_type__a_positive))

jasper__has_blood_type__dea_one_point_one_negative = Node('Instance Connection', name='jasper has blood type B-')
graph.create(Relationship(jasper__has_blood_type__dea_one_point_one_negative, IS_A, has_blood_type))
graph.create(Relationship(jasper,                     IS_THE_SOURCE_OF, jasper__has_blood_type__dea_one_point_one_negative))
graph.create(Relationship(dea_one_point_one_negative, IS_THE_TARGET_OF, jasper__has_blood_type__dea_one_point_one_negative))

jacoby__has_blood_type__invertebrate_blood = Node('Instance Connection', name='jacoby has blood type B-')
graph.create(Relationship(jacoby__has_blood_type__invertebrate_blood, IS_A, has_blood_type))
graph.create(Relationship(jacoby,             IS_THE_SOURCE_OF, jacoby__has_blood_type__invertebrate_blood))
graph.create(Relationship(invertebrate_blood, IS_THE_TARGET_OF, jacoby__has_blood_type__invertebrate_blood))

jack_left_kidney__kidney_is_inside__jack = Node('Instance Connection', name="jack's left kidney is inside jack")
graph.create(Relationship(jack_left_kidney__kidney_is_inside__jack, IS_A, kidney_is_inside))
graph.create(Relationship(jack_left_kidney, IS_THE_SOURCE_OF, jack_left_kidney__kidney_is_inside__jack))
graph.create(Relationship(jack,             IS_THE_TARGET_OF, jack_left_kidney__kidney_is_inside__jack))

jill_right_kidney__kidney_is_inside__jill = Node('Instance Connection', name="jill's right kidney is inside jill")
graph.create(Relationship(jill_right_kidney__kidney_is_inside__jill, IS_A, kidney_is_inside))
graph.create(Relationship(jill_right_kidney, IS_THE_SOURCE_OF, jill_right_kidney__kidney_is_inside__jill))
graph.create(Relationship(jill,              IS_THE_TARGET_OF, jill_right_kidney__kidney_is_inside__jill))

john_left_kidney__kidney_is_inside__john = Node('Instance Connection', name="john's left kidney is inside john")
graph.create(Relationship(john_left_kidney__kidney_is_inside__john, IS_A, kidney_is_inside))
graph.create(Relationship(john_left_kidney, IS_THE_SOURCE_OF, john_left_kidney__kidney_is_inside__john))
graph.create(Relationship(john,             IS_THE_TARGET_OF, john_left_kidney__kidney_is_inside__john))

john_right_kidney__kidney_is_inside__john = Node('Instance Connection', name="john's right kidney is inside john")
graph.create(Relationship(john_right_kidney__kidney_is_inside__john, IS_A, kidney_is_inside))
graph.create(Relationship(john_right_kidney, IS_THE_SOURCE_OF, john_right_kidney__kidney_is_inside__john))
graph.create(Relationship(john,              IS_THE_TARGET_OF, john_right_kidney__kidney_is_inside__john))

jane_left_kidney__kidney_is_inside__jane = Node('Instance Connection', name="jane's left kidney is inside jane")
graph.create(Relationship(jane_left_kidney__kidney_is_inside__jane, IS_A, kidney_is_inside))
graph.create(Relationship(jane_left_kidney, IS_THE_SOURCE_OF, jane_left_kidney__kidney_is_inside__jane))
graph.create(Relationship(jane,             IS_THE_TARGET_OF, jane_left_kidney__kidney_is_inside__jane))

jane_right_kidney__kidney_is_inside__jane = Node('Instance Connection', name="jane's right kidney is inside jane")
graph.create(Relationship(jane_right_kidney__kidney_is_inside__jane, IS_A, kidney_is_inside))
graph.create(Relationship(jane_right_kidney, IS_THE_SOURCE_OF, jane_right_kidney__kidney_is_inside__jane))
graph.create(Relationship(jane,              IS_THE_TARGET_OF, jane_right_kidney__kidney_is_inside__jane))

jasper_left_kidney__kidney_is_inside__jasper = Node('Instance Connection', name="jasper's left kidney is inside jasper")
graph.create(Relationship(jasper_left_kidney__kidney_is_inside__jasper, IS_A, kidney_is_inside))
graph.create(Relationship(jasper_left_kidney, IS_THE_SOURCE_OF, jasper_left_kidney__kidney_is_inside__jasper))
graph.create(Relationship(jasper,             IS_THE_TARGET_OF, jasper_left_kidney__kidney_is_inside__jasper))

jasper_right_kidney__kidney_is_inside__jasper = Node('Instance Connection', name="jasper's left kidney is inside jasper")
graph.create(Relationship(jasper_right_kidney__kidney_is_inside__jasper, IS_A, kidney_is_inside))
graph.create(Relationship(jasper_right_kidney, IS_THE_SOURCE_OF, jasper_right_kidney__kidney_is_inside__jasper))
graph.create(Relationship(jasper,              IS_THE_TARGET_OF, jasper_right_kidney__kidney_is_inside__jasper))

jack__is_friends_with__jill = Node('Instance Connection', name='jack is friends with jill')
graph.create(Relationship(jack__is_friends_with__jill, IS_A, is_friends_with))
graph.create(Relationship(jack, IS_THE_SOURCE_OF, jack__is_friends_with__jill))
graph.create(Relationship(jill, IS_THE_TARGET_OF, jack__is_friends_with__jill))

jill__is_friends_with__john = Node('Instance Connection', name='jill is friends with john')
graph.create(Relationship(jill__is_friends_with__john, IS_A, is_friends_with))
graph.create(Relationship(jill, IS_THE_SOURCE_OF, jill__is_friends_with__john))
graph.create(Relationship(john, IS_THE_TARGET_OF, jill__is_friends_with__john))

john__is_friends_with__jane = Node('Instance Connection', name='john is friends with jane')
graph.create(Relationship(john__is_friends_with__jane, IS_A, is_friends_with))
graph.create(Relationship(john, IS_THE_SOURCE_OF, john__is_friends_with__jane))
graph.create(Relationship(jane, IS_THE_TARGET_OF, john__is_friends_with__jane))

jack__is_friends_with__jasper = Node('Instance Connection', name='jack is friends with jasper')
graph.create(Relationship(jack__is_friends_with__jasper, IS_A, is_friends_with))
graph.create(Relationship(jack,   IS_THE_SOURCE_OF, jack__is_friends_with__jasper))
graph.create(Relationship(jasper, IS_THE_TARGET_OF, jack__is_friends_with__jasper))

jack__is_friends_with__jacoby = Node('Instance Connection', name='jack is friends with jacoby')
graph.create(Relationship(jack__is_friends_with__jacoby, IS_A, is_friends_with))
graph.create(Relationship(jack,   IS_THE_SOURCE_OF, jack__is_friends_with__jacoby))
graph.create(Relationship(jacoby, IS_THE_TARGET_OF, jack__is_friends_with__jacoby))


draw(graph, labels=['Instance Connection'])

In [13]:
draw(graph, labels=['Instance Element'])

In [14]:
draw(graph, relax_gray_relationships=True)

## Voila! A fully-queryable metamodel/model/instance database!

##### Queryable?

Neo4j queries are performed with the [Cypher Query Language](http://neo4j.com/docs/stable/cypher-query-lang.html).

Much of a Cypher query looks like you're drawing a graph between `(nodes) -[relationships]-> (and other nodes)`, with a few `WHERE` and `AS` keywords thrown in.

Both nodes and relationships can have labels and properties, so we add those labels and properties to the "drawings" in our queries, in order to specify which nodes and relationships we're looking for.

In [15]:
# Refer to a node: (variable_name)
graph.cypher.execute("MATCH ({name:'jack'}) -[:IS_A]-> (jack_type) RETURN jack_type.name AS `Jack's Type`")

   | Jack's Type
---+-------------
 1 | human      

In [16]:
# Refer to multiple nodes in the same way
graph.cypher.execute("MATCH (humans) -[:IS_A]-> ({name: 'human'}) RETURN humans.name AS `Humans`")

   | Humans
---+--------
 1 | jane  
 2 | john  
 3 | jill  
 4 | jack  

In [17]:
# Bi-directional relationships: -[]-
graph.cypher.execute("MATCH ({name:'jack'}) -[*1..2]- (neighbors) RETURN DISTINCT neighbors.name AS `Jack's Nearby Nodes`")

    | Jack's Nearby Nodes              
----+-----------------------------------
  1 | human                            
  2 | animal                           
  3 | animal generalizes human         
  4 | human implements vertebrate      
  5 | jill                             
  6 | john                             
  7 | jane                             
  8 | jack has blood type A+           
  9 | has blood type                   
 10 | a+                               
 11 | jack's left kidney is inside jack
 12 | kidney is inside                 
 13 | jack's left kidney               
 14 | jack is friends with jill        
 15 | is friends with                  
 16 | jack is friends with jasper      
 17 | jasper                           
 18 | jack is friends with jacoby      
 19 | jacoby                           

Node properties, relationship types, and path length allow us to ask much more complex and useful questions, too.

##### Like what?

Let's ask our database some questions to learn about Jack's situation:

* What is Jack's blood type?
* Who has Jack's blood type?
* Who has a spare kidney?
* Who has Jack's blood type and a spare kidney?
* Who is Jack friends with?
* Who can Jack become friends with?
* Who can Jack become friends with that has Jack's blood type and a spare kidney?
* Whose blood type do we not know?

In [18]:
query = """MATCH
(jack {name:"jack"})          -[:IS_THE_SOURCE_OF]-> (jack__blood_type_connection),
(jack_blood_type)             -[:IS_THE_TARGET_OF]-> (jack__blood_type_connection),
(jack__blood_type_connection)       -[:IS_A]->       ({name: "has blood type"})
RETURN
jack_blood_type.name AS `Jack's blood type`"""

graph.cypher.execute(query)

   | Jack's blood type
---+-------------------
 1 | a+               

In [19]:
query = """MATCH
(jack {name:"jack"})          -[:IS_THE_SOURCE_OF]-> (jack__blood_type_connection),
(jack_blood_type)             -[:IS_THE_TARGET_OF]-> (jack__blood_type_connection),
(jack__blood_type_connection)      -[:IS_A]->       ({name: "has blood type"}),
(others_with_jack_blood_type) -[:IS_THE_SOURCE_OF]-> () <-[:IS_THE_TARGET_OF]- (jack_blood_type)
RETURN
others_with_jack_blood_type.name AS `Others with Jack's blood type`"""

graph.cypher.execute(query)

   | Others with Jack's blood type
---+-------------------------------
 1 | jane                         

In [20]:
query = """MATCH
(others_with_a_kidney) -[:IS_THE_TARGET_OF]-> () -[:IS_A]-> ({name:"kidney is inside"})
WHERE
others_with_a_kidney.name <> "jack"
RETURN
others_with_a_kidney.name AS `Others with a kidney`,
count(*) AS `Number of kidneys` ORDER BY `Number of kidneys` DESC
"""

graph.cypher.execute(query)

   | Others with a kidney | Number of kidneys
---+----------------------+-------------------
 1 | jane                 |                 2
 2 | jasper               |                 2
 3 | john                 |                 2
 4 | jill                 |                 1

In [21]:
# TODO: check that they have multiple kidneys
query = """MATCH
(jack {name:"jack"})          -[:IS_THE_SOURCE_OF]-> (jack__blood_type_connection),
(jack_blood_type)             -[:IS_THE_TARGET_OF]-> (jack__blood_type_connection),
(jack__blood_type_connection)      -[:IS_A]->        ({name: "has blood type"}),
(others_with_jack_blood_type) -[:IS_THE_SOURCE_OF]-> () <-[:IS_THE_TARGET_OF]- (jack_blood_type)
WHERE
(others_with_jack_blood_type) -[:IS_THE_TARGET_OF]-> () -[:IS_A]-> ({name:"kidney is inside"})
RETURN
others_with_jack_blood_type.name AS `Others with Jack's blood type and a kidney`"""

graph.cypher.execute(query)

   | Others with Jack's blood type and a kidney
---+--------------------------------------------
 1 | jane                                      

In [22]:
query = """MATCH
({name:"jack"}) -[:IS_THE_SOURCE_OF]-> (friendships) -[:IS_A]-> ({name: "is friends with"}),
(friends)       -[:IS_THE_TARGET_OF]-> (friendships)
RETURN friends.name as `Jack's immediate friends`
"""

graph.cypher.execute(query)

   | Jack's immediate friends
---+--------------------------
 1 | jacoby                  
 2 | jasper                  
 3 | jill                    

In [23]:
# TODO: only traverse source/target relationships for friendship connections
query = """MATCH
({name:"jack"}) -[:`IS_THE_SOURCE_OF`|`IS_THE_TARGET_OF`*]- (friendships_of_friends:`Instance Connection`),
(friendships_of_friends)       -[:IS_A]->       ({name: "is friends with"}),
(friends)                -[:IS_THE_TARGET_OF]-> (friendships_of_friends)
RETURN DISTINCT friends.name as `Friends of Jack's friends`
"""

graph.cypher.execute(query)

   | Friends of Jack's friends
---+---------------------------
 1 | jill                     
 2 | john                     
 3 | jane                     
 4 | jasper                   
 5 | jacoby                   

In [24]:
# TODO: check that they have multiple kidneys
# TODO: only traverse source/target relationships for friendship connections
query = """MATCH
(jack {name:"jack"})           -[:IS_THE_SOURCE_OF]-> (jack__blood_type_connection),
(jack_blood_type)              -[:IS_THE_TARGET_OF]-> (jack__blood_type_connection),
(jack__blood_type_connection)        -[:IS_A]->       ({name: "has blood type"}),
(friends_with_jack_blood_type) -[:IS_THE_SOURCE_OF]-> () <-[:IS_THE_TARGET_OF]- (jack_blood_type),
(jack)   -[:`IS_THE_SOURCE_OF`|`IS_THE_TARGET_OF`*]-  (friendships_of_friends:`Instance Connection`),
(friendships_of_friends)             -[:IS_A]->       ({name: "is friends with"}),
(friends_with_jack_blood_type) -[:IS_THE_TARGET_OF]-> (friendships_of_friends)
WHERE
(friends_with_jack_blood_type) -[:IS_THE_TARGET_OF]-> () -[:IS_A]-> ({name:"kidney is inside"})
RETURN
friends_with_jack_blood_type.name AS `Friends of Jack's friends with his blood type and a kidney`"""

graph.cypher.execute(query)

   | Friends of Jack's friends with his blood type and a kidney
---+------------------------------------------------------------
 1 | jane                                                      

In [25]:
query = """MATCH
(humans) -[:IS_A]-> ({name: 'human'})
WHERE
NOT (humans) -[:IS_THE_SOURCE_OF]-> () -[:IS_A]-> ({name: "has blood type"})
RETURN
humans.name AS `Humans with an unknown blood type`"""

graph.cypher.execute(query)

   | Humans with an unknown blood type
---+-----------------------------------
 1 | john                             

##### So what?

We have identified a recommendation ("Jane is a viable kidney donor") and an unknown ("What is John's blood type?").

##### This approach seems overly-complex!

Defining one node and two relationships for each connection does seem unnecessary, but we must define relationships **between connections** as well.

##### Why?

With this implementation, the graph database contains all of the information required to support a modeling user interface.

*From this point on, I'll use the word "user" to mean modelers, subject matter experts, and project managers, but not software developers.*

User views of the project model/data should never look like the crowded visualizations above. Users should only interact with diagrams that focus the visualizations and constrain their actions to legal ones.

User actions (creating/editing/deleting an element or connection) should be limited by:

* the diagram's subset of nodes
* the diagram's subset of relationships
* the existing project data in the database
* the model's definitions of what elements/connections can be made
* the metamodel's definitions of what elements/connections can be made

The metamodel/model/instance graph database implementation described on this page provides a technologically-minimalist modeling tool with the logical complexity to express any project's model/data and contrain any diagramming interface's user actions.

I look forward to implementing such a diagramming interface and asking the graph database questions like:

* What are all undefined properties for elements in this diagram?
* For the currently-selected element, what are all elements to which it can connect?
* What are all the possible "Connect X to Y with Z" buttons for the toolbar of this diagram?

**Thanks for reading! I'd love to hear your thoughts/questions about anything on this page.**

\- Jan